In [1]:
from _shared import *
from tqdm.notebook import tqdm

## Задача 1. Количество параллельных экспериментов

Сколько несовместных независимых экспериментов можно запустить одновременно, если за время эксперимента собираем 10000 наблюдений?

Если решаем запустить 10 экспериментов, то на каждый эксперимент можно выделить по 1000 наблюдений, размер групп будет равен 500.

Параметры экспериментов:
- проверяем гипотезу о равенстве средних;
- уровень значимости — 0.05;
- допустимая вероятность ошибки II рода — 0.1;
- ожидаемый эффект — увеличение значений на 3%;
- способ добавления эффекта в синтетических А/Б экспериментах — умножение на константу.

Будем считать, что распределение измеряемых величин является нормальным распределением со средним 100 и стандартным отклонением 10.

В качестве ответа введите максимально возможное количество экспериментов, которое можно запустить с указанными выше параметрами.

In [2]:
obs_num = 10_000
test_ = stats.ttest_ind

alpha_ = 0.05
beta_ = 0.1
effect_ = +0.03

mean_ = 100.0
std_ = 10.0

In [3]:
# df = pd.DataFrame(np.random.normal(loc=mean_, scale=std_, size=obs_num), columns=['metric_a']) #Let index be the user_id.
# df['metric_b'] = df['metric_a'] * (1 + effect_)

# Number of test equals half the number of observation divided by required sample size (since we need 2 groups).
res_1 = np.floor(obs_num / get_sample_size_rel(mu=mean_, std=std_, eff=effect_, alpha=alpha_, beta=beta_)) / 2
res_1

21.0

In [4]:
def check_experiment_design(number_of_experiments, N=10_000):
    group_size = int((obs_num / number_of_experiments) / 2)
    aa_res, ab_res = [], []
    
#    for _ in tqdm(range(N)):
#         for i in range(number_of_experiments):
#             a_beg = i * group_size * 2
#             a_end = a_beg + group_size - 1
#             b_beg = a_end + 1
#             b_end = b_beg + group_size - 1

#             aa_res.append(test_(metric_a[a_beg:a_end], metric_b[b_beg:b_end]).pvalue)
#             ab_res.append(test_(metric_a[a_beg:a_end], metric_b_eff[b_beg:b_end]).pvalue)

    for _ in tqdm(range(N)):
        metric_a, metric_b = np.random.normal(loc=mean_, scale=std_, size=(2, group_size))
        metric_b_eff = metric_b * (1.0 + effect_)

        aa_res.append(test_(metric_a, metric_b).pvalue)
        ab_res.append(test_(metric_a, metric_b_eff).pvalue)

    return ((np.array(aa_res) < alpha_).mean(), (np.array(ab_res) >= alpha_).mean())


res_string = '{:.0f}: I order: {:.4f} [{}]; II order: {:.4f} [{}]'

In [5]:
for n in (res_1-1, res_1, res_1+1):
    p_i, p_ii = check_experiment_design(n, N=100_000)
    print(res_string.format(n, p_i, p_i < alpha_, p_ii, p_ii < beta_))

  0%|          | 0/100000 [00:00<?, ?it/s]

20: I order: 0.0482 [True]; II order: 0.0909 [True]


  0%|          | 0/100000 [00:00<?, ?it/s]

21: I order: 0.0501 [False]; II order: 0.1055 [False]


  0%|          | 0/100000 [00:00<?, ?it/s]

22: I order: 0.0499 [True]; II order: 0.1188 [False]


In [6]:
# Solution.
import numpy as np
from scipy import stats
from tqdm.notebook import tqdm

# параметры эксперимента
total_size = 10000
mean_ = 100
std_ = 10
effect = 0.03
alpha = 0.05
beta = 0.1


def estimate_sample_size(effect, std, alpha, beta):
    """Оценка необходимого размер групп."""
    t_alpha = stats.norm.ppf(1 - alpha / 2, loc=0, scale=1)
    t_beta = stats.norm.ppf(1 - beta, loc=0, scale=1)
    var = 2 * std ** 2
    sample_size = int((t_alpha + t_beta) ** 2 * var / (effect ** 2))
    return sample_size


# оценим необходимый размер групп
sample_size = estimate_sample_size(effect * 100, 10, alpha, beta)
print(f'sample_size = {sample_size}')
# вычислим количество экспериментов
count_exp = total_size / (sample_size * 2)
print(f'count_exp = {count_exp:0.1f}')


def estimate_ci_bernoulli(p, n, alpha=0.05):
    """Доверительный интервал для Бернуллиевской случайной величины."""
    t = stats.norm.ppf(1 - alpha / 2, loc=0, scale=1)
    std_n = np.sqrt(p * (1 - p) / n)
    return p - t * std_n, p + t * std_n

# Проверим, что при 21 эксперимента ошибки контролируются на заданных уровнях, а при 22 экспериментах нет.

for count_exp in [21, 22]:
    errors_aa = []
    errors_ab = []
    sample_size = int(total_size / (int(count_exp) * 2))
    for _ in tqdm(range(10000)):
        a, b = np.random.normal(mean_, std_, (2, sample_size,))
        b_effect = b * (1 + effect)
        errors_aa.append(stats.ttest_ind(a, b).pvalue < alpha)
        errors_ab.append(stats.ttest_ind(a, b_effect).pvalue >= alpha)

    estimated_first_type_error = np.mean(errors_aa)
    estimated_second_type_error = np.mean(errors_ab)
    ci_first = estimate_ci_bernoulli(estimated_first_type_error, len(errors_aa))
    ci_second = estimate_ci_bernoulli(estimated_second_type_error, len(errors_ab))
    print(f'count_exp = {count_exp}')
    print(f'sample_size = {sample_size}')
    print(f'оценка вероятности ошибки I рода = {estimated_first_type_error:0.4f}')
    print(f'  доверительный интервал = [{ci_first[0]:0.4f}, {ci_first[1]:0.4f}]')
    print(f'оценка вероятности ошибки II рода = {estimated_second_type_error:0.4f}')
    print(f'  доверительный интервал = [{ci_second[0]:0.4f}, {ci_second[1]:0.4f}]')


sample_size = 233
count_exp = 21.5


  0%|          | 0/10000 [00:00<?, ?it/s]

count_exp = 21
sample_size = 238
оценка вероятности ошибки I рода = 0.0477
  доверительный интервал = [0.0435, 0.0519]
оценка вероятности ошибки II рода = 0.0996
  доверительный интервал = [0.0937, 0.1055]


  0%|          | 0/10000 [00:00<?, ?it/s]

count_exp = 22
sample_size = 227
оценка вероятности ошибки I рода = 0.0505
  доверительный интервал = [0.0462, 0.0548]
оценка вероятности ошибки II рода = 0.1221
  доверительный интервал = [0.1157, 0.1285]


## Задача 2. Количество параллельных экспериментов — 2

Задача похожа на предыдущую, только теперь __решение принимается не независимо для каждого эксперимента.__
Например, у нас есть 5 текстов для маркетинговой рассылки, хотим проверить, какой эффективнее работает и работает ли вообще.

__Алгоритм будет следующий:__
1. Формируем непересекающиеся контрольные и экспериментальные группы для каждого из 5 вариантов.
2. Проводим параллельно 5 экспериментов.
3. С помощью метода Холма определяем, в каких экспериментах были статистически значимые отличия.
4. Если значимых отличий не обнаружено, то говорим, что эффекта нет, все варианты отклоняем.
5. Если значимые отличия обнаружены, то из вариантов со значимым эффектом выбираем вариант с наименьшим значением p-value, будем использовать его.

Будем считать, что __совершена ошибка I рода__, если найдены значимые отличия, когда на самом деле их не было ни в одном из вариантов.

Будем считать, что __совершена ошибка II рода__, если:
- либо не найдено значимых отличий, когда на самом деле они были;
- либо выбранный для дальнейшего использования вариант на самом деле был без эффекта, при этом были варианты с эффектом.

__Параметры экспериментов:__
- проверяем гипотезу о равенстве средних;
- уровень значимости — 0.05;
- допустимая вероятность ошибки II рода — 0.1;
- ожидаемый эффект — увеличение значений на 3%;
- способ добавления эффекта в синтетических А/Б экспериментах — умножение на константу.

Замечание: при оценке вероятности ошибки II рода нужно рассматривать худший сценарий, когда эффект есть только в одном из экспериментов. Чем в большем количестве экспериментов будет эффект, тем меньше будет вероятность ошибки II рода.

Будем считать, что распределение измеряемых величин является нормальным распределением со средним 100 и стандартным отклонением 10.

В качестве ответа введите максимально возможное количество экспериментов, которое можно запустить с указанными выше параметрами.



In [7]:
def bonferroni_method(pvalues, alpha=0.05):
    """
    Carries out Bonferroni correction.

    pvalues - list of pvalues.
    alpha -  level of significance.
    return - array of 0/1 flags indicating presence/absence of effect.
    """
    pvalues = np.array(pvalues)
    
    return np.array(pvalues < (alpha / pvalues.shape[0])).astype(int)


def holm_method(pvalues, alpha=0.05):
    """
    Carries out Holm method correction.

    pvalues - list of pvalues.
    alpha -  level of significance.
    return - array of 0/1 flags indicating presence/absence of effect.
    """
    n = len(pvalues)
    
    alphas = alpha / np.arange(n, 0, -1) #Array of alpha divided by indices.
    sorted_indices = np.argsort(pvalues)
    res = np.zeros(n)
    
    for (i, pv_i) in enumerate(sorted_indices):
        if pvalues[pv_i] < alphas[i]:
            res[pv_i] = 1
        else:
            break

    return res.astype(int)

In [8]:
correction_ = holm_method